In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19596888
paper_name = 'lis_bobek_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [14]:
original_data = pd.read_excel('raw_data/Lis_2009.xlsx', sheet_name='Log2 Ratio')

In [15]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4630 x 4


In [16]:
original_data.head()

,ORF,0h,24h,48h
0,YAL002W,0.129931,0.242908,0.774877
1,YAL004W,0.208012,-0.046441,-0.083625
2,YAL005C,0.318190,-0.077967,-0.287597
3,YAL007C,0.141652,0.038926,0.027181
4,YAL008W,0.123745,-0.398342,0.007062


In [17]:
original_data['orf'] = original_data['ORF'].astype(str)

In [18]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [19]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [20]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, 0h, 24h, 48h, orf]
Index: []


In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['0h','24h','48h']].copy()

In [23]:
# log2 control/treatment must be reversed
original_data = -original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [24]:
original_data = original_data.groupby(original_data.index).mean()

In [25]:
original_data.shape

(4575, 3)

In [26]:
# Normalize by 0h
original_data = original_data.sub(original_data['0h'], axis=0)

In [28]:
original_data.drop(columns=['0h'], inplace=True)

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [4924, 5255]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,4924,5255
data_type,value,value
orf,,
YAL002W,-0.112977,-0.644946
YAL004W,0.254453,0.291638
YAL005C,0.396157,0.605787
YAL007C,0.102726,0.114472
YAL008W,0.522088,0.116683


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 16


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4924,5255
,data_type,value,value
gene_id,orf,,
2,YAL002W,-0.112977,-0.644946
1863,YAL004W,0.254453,0.291638
4,YAL005C,0.396157,0.605787
5,YAL007C,0.102726,0.114472
6,YAL008W,0.522088,0.116683


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4924,5255,4924,5255
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,-0.112977,-0.644946,-0.465234,-1.205218
1863,YAL004W,0.254453,0.291638,0.345901,0.319704
4,YAL005C,0.396157,0.605787,0.658725,0.831194
5,YAL007C,0.102726,0.114472,0.010950,0.031247
6,YAL008W,0.522088,0.116683,0.936727,0.034848


# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 19596888...
Inserting the new data...


100%|██████████| 2/2 [00:16<00:00,  8.29s/it]

Updating the data_modified_on field...
